# Project Title
### Data Engineering Capstone Project

#### Project Summary
This project will answer questins related to US immigration.
Data sources were provided by Udacity which are:
- 94 Immigration Data
- U.S. City Demographic Data
- Airport Code Table

Technologies used are Spark for processing .SAS files.
Pandas for processing .csv files

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
from pyspark.sql import SparkSession, SQLContext, GroupedData
import pyspark.sql.functions as f

### Step 1: Scope the Project and Gather Data

#### Scope 
The purpose of this project is to create Dimention and Fact tables from raw data provided from 3 different sources.
The final fact table will show the number of immigrants per country and from whcich airport they came and to which state they went.

#### Describe and Gather Data 
- **I94 Immigrantion Data**: comes form the US National Tourism and Trade Office.
- **U.S City Demographic Data**: comes from OpenSoft.
- **Airport Code Table: comes** from DataHub.io

### Step 2: Explore and Assess the Data

#### Cleaning Steps

* Immigration:
    - Read files per month into a Spark DataFrame.
    - Filter the missing values.
    - Generate the corresponding names and codes per state and city using UDF.
    - Write the Dimesion Table into Parquet.

* Demographics:
    - Group race values into shorter values.
    - Calculate the average and sum per State
    - Calculate percentages per columns.
    - Calculate percentages per race.
    - Write the Dimension Table into a CSV file.
    
* Airports:
    - Filter USA data.
    - Transform data types for several columns.
    - Write the Dimension Table into a CSV file.
    

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

The chosen schema for modeling is Star Schema.

**Star Schema**

1. Dimension Tables
    * Immigration
    * Demographics
    * Airports
2. Fact Table
    * Fact_Immigration

<img src="img.png">

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

    1. Dimension tables will be populated with the raw data after cleaning.
    2. A TempView will be created for every table using Spaark.
    3. Spark SQL will be used to create the Fact Table.
    4. Fact table generated will be written in a parquet file.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Note: Run etl.py before running below cells

In [ ]:
#create Spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

sqlContext = SQLContext(spark)
sqlContext.setConf("spark.sql.autoBroadcastJoinThreashold", "0")

In [ ]:
# Read every Dimension table
demographicsDF = spark.read.csv("us_demographics.csv",header=True)
airportsDF = spark.read.csv("us_airports.csv",header=True)
immigrationDF = spark.read.parquet("immigration_data")

In [ ]:
# For time processing purposes, I will be using only the first 2 months in the immigration dataframe
immigrationDF = immigrationDF.filter(f.col("month")<f.lit(2))

In [ ]:
# Create Dimension TempViews
immigrationDF.createOrReplaceTempView("immigration")
demographicsDF.createOrReplaceTempView("demographics")
airportsDF.createOrReplaceTempView("airports")

In [ ]:
# Running SQL Procedure to get the fact table dataframe
fact_table = spark.sql("""
    SELECT
        immigration.year,
        immigration.month,
        immigration.origin_country,
        immigration.State,
        immigration.state_code,
        COUNT(immigration.state_code) as number_immigrants,
        demographics.median_age,
        demographics.percentage_male,
        demographics.percentage_female,
        demographics.percentage_veterans,
        demographics.percentage_foreign_born,
        demographics.Afroamerican as percentage_afroamerican,
        demographics.Asian as percentage_asian,
        demographics.Latino as percentage_latino,
        demographics.Native as percentage_native,
        demographics.White as percentage_white,
        airports.name as airport_name,
        airports.x_coordinate,
        airports.y_coordinate
    FROM immigration
    JOIN demographics ON demographics.state_code=immigration.state_code
    JOIN airports ON airports.state_code=immigration.state_code
    WHERE airports.type='large_airport'
    GROUP BY 1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19
    ORDER BY 1,2,3,4
""").cache()

In [ ]:
# Count rows in dataframe
fact_table.count()

In [ ]:
fact_table.show()

In [ ]:
# Write Fact Table in Parquet format
fact_table.write.mode('overwrite').parquet("fact_table")

#### 4.2 Data Quality Checks
 
Run Quality Checks

In [ ]:
# Data quality checks
fact_table = spark.read.parquet("fact_table")
fact_table.createOrReplaceTempView("fact_table")

In [ ]:
# Check null values per column
null_values_test = fact_table.select(f.isnull('year'),\
                  f.isnull('month'),\
                  f.isnull('origin_country'),\
                  f.isnull('state_code'),\
                  f.isnull('State')).dropDuplicates()
null_values_test.show()

In [ ]:
# Check if there are immigrants that were no copied from the original immigrants dimensional table
immigrants_check_integrity = spark.sql("""
    SELECT I.year, I.month, I.State, I.state_code
    FROM (
        SELECT year, month, State, state_code, COUNT(state_code) as number_immigrants
        FROM immigration
        GROUP BY 1,2,3,4
        ORDER BY 1,2,3,4
    ) I, 
    (
        SELECT year, month, State, state_code, SUM(number_immigrants) as sum_immigrants
        FROM fact_table
        GROUP BY 1,2,3,4
        ORDER BY 1,2,3,4
    ) F
    WHERE I.year=F.year and I.month=F.month and I.state_code=F.state_code and I.number_immigrants>F.sum_immigrants
""")
immigrants_check_integrity.show()

In [ ]:
# Check Sum for number of immigrants equal to 0
immigrants_check_sum = spark.sql("""
    SELECT A.State, A.state_code, A.sum_immigrants
    FROM (
        SELECT State, state_code, SUM(number_immigrants) as sum_immigrants
        FROM fact_table
        GROUP BY 1,2
        ORDER BY 1,2
    ) A
    WHERE A.sum_immigrants=0
""")
immigrants_check_sum.show()

#### 4.3 Data dictionary 

**Dimension Tables**
```
immigrant_df:
    ID: id of immigrant
    Year: year
    Month: month
    Origin_country: immigrant's original country
    City_code: city port code
    State_code: state code of the city
    State: name of the state
    City: name of the city
    
demographics_df:
    State_code: state code of the city
    State: name of the state
    Median_age: median age
    Percentage_male: male population
    Percentage_female: female population
    Percentage_veterans: veterans population
    Percentage_foreign_born: male population
    Percentage_native: male population
    Percentage_white: white population
    Percentage_afroamerican: afroamerican population
    Percentage_latino: latino population
    Percentage_asian: asian population
    
    
airport_df:
    Ident: ID to identify an airport
    Type: Size and type of an airport
    Name: The name of an airport 
    Elevation_ft: The distance above sea level
    Country: Country of an airport
    State_code: State code of the city an airport is in
    City_code: City code that an airport is in
    Municipality: The municipality of that city
    X_coordinate: Longitude of the airport
    Y_coordinate: Latitude of the airport
    
```

**Fact Table**
```
fact_table:
    Year: year
    Month: month
    Origin_country: immigrant's original country
    Number_immigrants: total number of immigrants per state
    State_code: state code of the city
    State: name of the state
    Median_age: median age
    Percentage_male: male population
    Percentage_female: female population
    Percentage_veterans: veterans population
    Percentage_foreign_born: male population
    Percentage_native: male population
    Percentage_white: white population
    Percentage_afroamerican: afroamerican population
    Percentage_latino: latino population
    Percentage_asian: asian population
    Airport_name: The name of an airport
    X_coordinate: Longitude of the airport
    Y_coordinate: Latitude of the airport
```

#### Step 5: Complete Project Write Up
**Spark** was chosen for this project because of it's capablity of processing Big Data, easily scalable by adding working nodes,and it integrates perfectly with Cloud storages such as AWS S3 and warehouses like AWS Redshift.
**Pandas** was chosen here since we're familiar with it and other resources were small compared to .SAS file, and it can be handeled using pandas.

- **Propose how often the data should be updated and why**
All computed results were grouped by month.
So it makes sense to update it monthly.

* Write a description of how you would approach the problem differently under the following scenarios:

 - **The data was increased by 100x**
 Pandas won't be able to handle the load. I will switch to Spark and add more node as needed.
 
 - **The data populates a dashboard that must be updated on a daily basis by 7am every day**
 Apache Airflow is perfect for the job. I will use it to schedule tasks to run everyday.
 - **The database needed to be accessed by 100+ people**
 Hosting the database on a cloud based solution will help. That will insure availability to all users and we will pay for what's needed.
 